In [2]:
from pyzbar.pyzbar import decode
from pdf2image import convert_from_path
import os
import numpy as np

In [15]:
# aqui buscamos dentro do caminho somente os arquivos .pdf, e no caso so permitira dentro da lista arquivos que sejam .pdf
pdfs = [i for i in os.listdir() if '.pdf' in i]
pdf_path = pdfs[0]
display(pdf_path)

'DAS-PAGO.pdf'

In [20]:
# aqui vamos converter um arquivo pdf em formato de imagem
pages = convert_from_path(pdf_path)
img = pages[0]

In [ ]:
# decodificando foto e pegando os dados
# podemos verificar o tipo do código de barras, usaremos .type
detected_datas = decode(img)
if not detected_datas:
    return False
else:
    for barcode in detected_datas:
        barcode = detected_datas[1]
        if barcode.data != '' and barcode.type == 'I25':
            barcode_ = barcode.data.decode('utf-8') #retornar a uma string do python
